# Convertible Bond Valuation - Convergence Analysis

Value a Convertible Bond using a Tree-Based model that incorporates the conversion option as well as the call and puts.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.bonds import *
from financepy.market.curves import *

## Creating the Convertible Bond

We set the maturity date, annualised coupon, coupon annual_frequency and convention for calculating accrued interest

In [3]:
maturity_dt = Date(20, 3, 2023)
coupon = 0.0
freq_type = FrequencyTypes.SEMI_ANNUAL
accrualBasis = DayCountTypes.ACT_365F

We need to set the face amount and the conversion ratio. This is the number of shares you can convert to for that face amount.

In [4]:
face = 100.0
conversion_ratio = 5.0  # Number of shares for face amount

We can also delay the start of the conversion feature using the start convert date

In [5]:
start_convert_dt = Date(20, 12, 2019)

We can also set the call schedule - the price and date on which the issuer can call back the bond. This is given with reference to a price of 100. Making the call price very high means it will never be called. So the call has no impact.

In [6]:
call_price = 120 # Make call uneconomic so it does not impact pricing
call_dts = [Date(20, 3, 2020), Date(20, 3, 2021), Date(20, 3, 2022)]
call_prices = np.array([call_price, call_price, call_price])

We can also set the put schedule - the price and date on which the owner can put back the bond to the issuer. This is given with reference to a price of 100. Making the put price very low means it will never be put.

In [7]:
putPrice = 0.0 # Make call uneconomic so it does not impact pricing
put_dts = [Date(20, 3, 2020), Date(20, 3, 2021), Date(20, 3, 2022)]
put_prices = np.array([putPrice, putPrice, putPrice])

At this point we can create our bond.

In [8]:
bond = BondConvertible(maturity_dt, coupon, freq_type, start_convert_dt, conversion_ratio, call_dts, call_prices, put_dts, put_prices, accrualBasis)

# Valuation

### No Calls and Puts

We need to specify the valuation date

In [9]:
settle_dt = Date(31, 12, 2019)

The model allows a discrete dividend schedule. We can use the FinSchedule class to do this.

In [10]:
dividend_schedule = Schedule(settle_dt, maturity_dt).schedule_dts()[1:]
dividend_yields = [0.01] * len(dividend_schedule)

We then need to set up the market inputs. 

In [11]:
stock_price = 20
stock_volatility = 0.30 # 30 percent
rate = 0.05
discount_curve = DiscountCurveFlat(settle_dt, rate, FrequencyTypes.ANNUAL)
credit_spread = 0.00
recovery_rate = 0.40 # 40 percent

The model is a binomial tree but you don't need to enter any tree details unless you want to as it defaults to 100 steps per year

We can now peform the valuation

In [12]:
bond.value(settle_dt, stock_price, stock_volatility, dividend_schedule, dividend_yields,
           discount_curve, credit_spread, recovery_rate)

{'cbprice': np.float64(109.73448110070586),
 'bond': np.float64(85.46491653997575),
 'delta': np.float64(2.9878835339742738),
 'gamma': np.float64(1.001353265428514),
 'theta': np.float64(109.31345846037485)}

'cbrice' is the bond price. 'bond' is the price of the bond alone without any conversion and without any puts or calls option

## Impact of Calls

In [13]:
call_price = 100000.0
call_prices = np.array([call_price, call_price, call_price])

Need to recreate the convertible bond.

In [14]:
bond = BondConvertible(maturity_dt, coupon, freq_type, start_convert_dt, conversion_ratio, call_dts, call_prices, put_dts, put_prices, accrualBasis)

In [15]:
bond.value(settle_dt, stock_price, stock_volatility, dividend_schedule, dividend_yields,
           discount_curve, credit_spread, recovery_rate)

{'cbprice': np.float64(111.4282617425936),
 'bond': np.float64(85.46491653997575),
 'delta': np.float64(3.4297479279504275),
 'gamma': np.float64(0.817378351037334),
 'theta': np.float64(125.92922284966545)}

## Stock Price Sensitivity

As the stock price goes up, the conversion options becomes more valuable

In [16]:
stock_prices = np.linspace(0,40,11)
print("%9s %12s %12s %12s"% ("StockPx", "CB Price", "Bond Price", "Delta"))

for stock_price in stock_prices:
    res = bond.value(settle_dt, stock_price, stock_volatility, dividend_schedule, dividend_yields,
                     discount_curve, credit_spread, recovery_rate)
    print("%9.2f %12.4f %12.4f %12.4f"% (stock_price, res['cbprice'], res['bond'], res['delta']))

  StockPx     CB Price   Bond Price        Delta
     0.00      85.4649      85.4649       0.0000
     4.00      85.4819      85.4649       0.0282
     8.00      86.4021      85.4649       0.5501
    12.00      90.6543      85.4649       1.5947
    16.00      99.1995      85.4649       2.6294
    20.00     111.4283      85.4649       3.4297
    24.00     126.3526      85.4649       3.9905
    28.00     143.1370      85.4649       4.3776
    32.00     161.2960      85.4649       4.6745
    36.00     180.4177      85.4649       4.8710
    40.00     200.0973      85.4649       4.9645


As expected the price rises with increasing stock price as the conversion option is more valuable

## Convergence Testing

Set up the bond description

In [17]:
settle_dt = Date(31, 12, 2019)

In [18]:
maturity_dt = Date(20, 3, 2023)
coupon = 0.0
freq_type = FrequencyTypes.SEMI_ANNUAL
accrualBasis = DayCountTypes.ACT_365F

call_price = 100000.0 # Make call uneconomic so it does not impact pricing
call_dts = [] # [Date(20, 3, 2020)]
call_prices = np.array([]) # [call_price]

putPrice = 0.0 # Make call uneconomic so it does not impact pricing
put_dts = [] # Date(20, 3, 2020)]
put_prices = np.array([]) # [putPrice]

face = 100.0
start_convert_dt = settle_dt
conversion_ratio = 5.0  # Number of shares for face amount

In [19]:
bond = BondConvertible(maturity_dt, coupon, freq_type, start_convert_dt, conversion_ratio, call_dts, call_prices, put_dts, put_prices, accrualBasis)

Set up market

In [20]:
stock_price = 20.0
stock_volatility = 0.10
rate = 0.05
discount_curve = DiscountCurveFlat(settle_dt, rate, FrequencyTypes.ANNUAL)
credit_spread = 0.0
recovery_rate = 0.0
dividend_schedule = [] # FinSchedule(settle_dt, maturity_dt).flows()[1:]
dividend_yields = [] # [0.0] * len(dividend_schedule)

How many tree steps per year do we need to give an accurate price ? Let us check.

In [21]:
stock_price = 25.0
stock_volatility = 0.30
stepsPerYearList = [100, 200, 400, 1000, 2000, 4000]
cbPrices = []

print("%9s %12s %12s %12s %12s"% ("Steps/Yr", "CB Price", "Bond Price", "Delta","Gamma"))
for num_steps_per_year in stepsPerYearList:
    res = bond.value(settle_dt, stock_price, stock_volatility, dividend_schedule, dividend_yields,
                     discount_curve, credit_spread, recovery_rate, num_steps_per_year)
    print("%9.2f %12.4f %12.4f %12.4f %12.4f"% (num_steps_per_year, res['cbprice'], res['bond'], res['delta'], res['gamma']))
    cbPrices.append(res['cbprice'])

 Steps/Yr     CB Price   Bond Price        Delta        Gamma
   100.00     132.7113      85.4649       4.1755       0.3364
   200.00     132.7074      85.4649       4.1761       0.5085
   400.00     132.7077      85.4649       4.1764       0.7534
  1000.00     132.7047      85.4649       4.1766       1.2406
  2000.00     132.7051      85.4649       4.1766       1.7907
  4000.00     132.7045      85.4649       4.1767       2.5689


In [22]:
plt.plot(stepsPerYearList,cbPrices);

Copyright (c) 2020 Dominic O'Kane